In [ ]:
import pandas as pd
import cv2
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from PIL import Image
import os
import pandas as pd
from torchvision import transforms
import torch.nn as nn

In [ ]:
anot_path = "./data/attention.csv"
images_path = "./data/datasets/"

gray_images_path = "./data/grayimage"

In [ ]:
df = pd.read_csv(anot_path)
df.columns = ['filename', 'score']
df['score'].value_counts()

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels_df.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        label = int(self.labels_df.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = CustomImageDataset(csv_file=anot_path, img_dir=images_path, transform=transform)
train_loader = DataLoader(dataset, batch_size=8, shuffle=False)

In [ ]:
model = models.efficientnet_b7(pretrained=True)
model.classifier = torch.nn.Identity()
model = model.to('cuda')
model.eval()

In [ ]:
features = []
labels = []

with torch.no_grad():
    for images, label in train_loader:
        images = images.to('cuda')
        outputs = model(images)
        features.append(outputs.cpu().numpy())
        labels.extend(label.cpu().numpy())

X_feat = np.concatenate(features, axis=0)
y_feat = np.array(labels)

print("Shape of X_feat: ", X_feat.shape)
print("Shape of y_feat: ", y_feat.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_feat, y_feat, test_size=0.2, random_state=42, stratify=y_feat
)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

params = {
    'C' : [0.1, 1, 10],
    'gamma' : ['scale', 0.01, 0.001],
    'kernel' : ['rbf', 'linear']
}

clf = GridSearchCV(SVC(), params, cv=5)
clf.fit(X_train, y_train)
train_acc = accuracy_score(y_train, y_train)
print(f"Train Acc : {train_acc*100:.2f}%")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_model(y_true, y_pred, class_names=None):
    """
    Evaluasi dan visualisasi hasil klasifikasi.

    Args:
        y_true (array): Label sebenarnya.
        y_pred (array): Label prediksi.
        class_names (list): Daftar nama kelas.
    """
    acc = accuracy_score(y_true, y_pred)
    print(f"🔍 Accuracy: {acc * 100:.2f}%\n")
    
    print("📊 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('🧩 Confusion Matrix')
    plt.show()


In [ ]:
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy : {acc * 100:.2f}%")

class_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4']

evaluate_model(y_test, y_pred, class_names=class_names)